# Dataset Creation from Train video
1. First check if the files are in Google Drive; if so, then just copy from there (maybe)
1. Generate JPG images from MP4 file and move to "original/images"
2. Generate TXT labels from train_mot_.txt and move to "valid/labels"
3. Separate into Train, Validation and Test datasets

TODO
- Consider resizing to 640x640 images (also see this post about possible problems with this)
    - https://github.com/ultralytics/ultralytics/issues/4510
- Remove valid images after copying to train,valid,test folders
- Add number of frames to dataset root folder to experiment with lower number of frames
- Improve the format of the image names (maybe)

# Pip packages

In [3]:
!pip install tqdm
!pip install ultralytics

  Using cached tqdm-4.66.1-py3-none-any.whl.metadata (57 kB)
Using cached tqdm-4.66.1-py3-none-any.whl (78 kB)
  Using cached pandas-2.0.3-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (18 kB)
  Using cached tzdata-2023.3-py2.py3-none-any.whl (341 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 44.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 65.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 57.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 50.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 78.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 17.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 36.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 47.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━

# Filenames definitions

In [7]:
import os 

data = '/work/marioeduardo-a/ftc'
images=os.path.join(data,"images")
labels=os.path.join(data,"labels")

images_original = os.path.join(images,'original')
labels_original = os.path.join(labels,'original')

images_train = os.path.join(images,'train')
labels_train = os.path.join(labels,'train')

images_valid = os.path.join(images,'valid')
labels_valid = os.path.join(labels,'valid')

images_test = os.path.join(images,'test')
labels_test = os.path.join(labels,'test')


video_train=os.path.join(data,'FTC-2024-data/Train/train.mp4')
labels_train_file=os.path.join(data,'FTC-2024-data/Train/train_gt_mot.txt')
# video_development=os.path.join(data,'FTC-2024-data/Development/development.mp4')
# video_test=os.path.join(data,'FTC-2024-data/Test/test.mp4')


## Generate original images from mp4 file

In [8]:
import cv2
import os
from tqdm import tqdm

# Copy images from drive ioriginalf they exist there
if not os.path.exists(images_original):
    os.makedirs(images_original)

    capture = cv2.VideoCapture(video_train)
    total_frames = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))
    # total_frames=100

    for f in tqdm(range(1,total_frames+1)):
        _,frame=capture.read()
        cv2.imwrite(os.path.join(images_original,str(f).zfill(6)+".jpg"),frame, [cv2.IMWRITE_JPEG_QUALITY, 100])
        # cv2.imwrite(newPath, frame, [cv2.IMWRITE_JPEG_QUALITY, 100])
        # cv2.imencode('.jpg', frame)[1].tofile(newPath)

    capture.release()
    #print("Last frame number: " + str(frames - 1))
    print("Images outputed: " + str(len(os.listdir(images_original))))


100%|█████████████████████████████████████████████████████████████████████████████| 10000/10000 [12:08<00:00, 13.73it/s]

Images outputed: 10000


# Labels

In [10]:
#extract labels into each frames to label_foler

if not os.path.exists(labels_original):
    video_width = 0
    video_height = 0
    cap = cv2.VideoCapture(video_train)
    if cap.isOpened():
        video_width  = cap.get(cv2.CAP_PROP_FRAME_WIDTH)   # float `width`
        video_height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)  # float `height`
    cap.release()

    print("Video width: " + str(video_width))
    print("Video height: " + str(video_height))

    
    os.makedirs(labels_original)


    print("Start translate labels into YOLO format.")
    print("Label source: " + labels_train_file)
    print("Label destnation: " + labels_original)

    current_frame = -1
    with open(labels_train_file) as f:
        current_file = 0
        for line in f:
            line = line.strip()
            frame, bid, top, left, width, height, _, _, _, _ = line.split(" ")
            frame, bid, left, top, width, height = int(frame), int(bid), float(left), float(top), int(width), int(height)
            center_x = left + width / 2
            center_y = top + height/ 2
            rel_center_x = center_x / video_width
            rel_center_y = center_y / video_height
            rel_width = width / video_width
            rel_height = height / video_height
            if frame != current_frame:
                if current_file != 0:
                    current_file.close()
                if frame == total_frames + 1:
                    break
                current_file = open(os.path.join(labels_original, str(frame).zfill(6) + ".txt"), 'w')
            #current_file.write("class_id center_x center_y bbox_width bbox_height\n")
            current_frame = frame
            current_file.write("0 " + str(rel_center_x) + " " + str(rel_center_y) + " " + str(rel_width) + " " + str(rel_height) + "\n")

        current_file.close()
    print("Translation finished!")
    print("Last frame number: " + str(current_frame))
    print("Labels outputed: " + str(len(os.listdir(labels_original))))



# Train Valid Test 

In [11]:
import os
import shutil
import random


if not os.path.exists(images_train):
    ids =[i for i in range(1,total_frames+1)]
    random.shuffle(ids)

    train_offset = int(0.75*total_frames)
    valid_offset = int(train_offset+0.15*total_frames)
    test_offset  = int(valid_offset+0.10*total_frames)
    print("train_offset:",train_offset)
    print("valid_offset:",valid_offset)
    print("test_offset:",test_offset)
    print()
    train_ids = ids[:train_offset]
    valid_ids = ids[train_offset:valid_offset]
    test_ids  = ids[valid_offset:]


    print('len(train):',len(train_ids))
    print('len(valid_ids):',len(valid_ids))
    print('len(train):',len(test_ids))
    print()
    # Create folders
    types=[images,labels]
    folders=["train","valid","test"]
    

    for t in types:
        for ff in folders:
            folder_fn = os.path.join(t,ff)
            if not os.path.exists(folder_fn):
                os.makedirs(folder_fn)

    #move training set to image_train_folder and label_train_folder
    for i in range(total_frames):
        if i<train_offset:
            destination_folder= "train"
        elif i<valid_offset:
            destination_folder= "valid"
        else:
            destination_folder= "test"
        destination_images=os.path.join(images,destination_folder)
        destination_labels=os.path.join(labels, destination_folder)

        source      = os.path.join(images_original,str(ids[i]).zfill(6)+".jpg")
        destination = os.path.join(destination_images,str(ids[i]).zfill(6)+".jpg")
        shutil.copy(source, destination)

        source      = os.path.join(labels_original,str(ids[i]).zfill(6)+".txt")
        destination = os.path.join(destination_labels,str(ids[i]).zfill(6)+".txt")

        shutil.copy(source, destination)


    # Validate sizes of folders
    #image count in colab vm
    print("Train images: ",len(os.listdir(images_train)))
    print("Train labels: ",len(os.listdir(labels_train)))
    print("Valid images: ",len(os.listdir(images_valid)))
    print("Valid labels: ",len(os.listdir(labels_valid)))
    print("Test images: ",len(os.listdir(images_test)))
    print("Test labels: ",len(os.listdir(labels_test)))
    

train_offset: 7500
valid_offset: 9000
test_offset: 10000

len(train): 7500
len(valid_ids): 1500
len(train): 1000

Train images:  7500
Train labels:  7500
Valid images:  1500
Valid labels:  1500
Test images:  1000
Test labels:  1000


In [33]:

for qq in ['test','train','valid']:
    for tt in ['images','labels']:
        print(tt,qq)
        # print('bef')
        !ls -1 /work/marioeduardo-a/ftc/{tt}/{qq} | head -n 3
        # print('aft')
    print()




images test
000002.jpg
000007.jpg
000008.jpg
ls: write error: Broken pipe
labels test
000002.txt
000007.txt
000008.txt
ls: write error: Broken pipe

images train
000001.jpg
000003.jpg
000004.jpg
ls: write error: Broken pipe
labels train
000001.txt
000003.txt
000004.txt
ls: write error: Broken pipe

images valid
000010.jpg
000019.jpg
000022.jpg
labels valid
000010.txt
000019.txt
000022.txt
ls: write error: Broken pipe

